In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dotenv import load_dotenv
import googlemaps
import pandas as pd
import os
import random
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
gmaps = googlemaps.Client(key=API_KEY)

# Define the Place ID
PLACE_ID = "ChIJj61dQgK6j4AR4GeTYWZsKWw"  # Example: Googleplex

# Fetch place details
place_details = gmaps.place(
    place_id=PLACE_ID,
    fields=[
        "geometry/location/lat",
        "plus_code",
        "current_opening_hours",
        "geometry/viewport/northeast/lng",
        "website",
        "reservable",
        "curbside_pickup",
        "address_component",
        "price_level",
        "geometry/viewport/northeast/lat",
        "type",
        "geometry/location/lng",
        "photo",
        "rating",
        "serves_breakfast",
        "serves_vegetarian_food",
        "editorial_summary",
        "secondary_opening_hours",
        "serves_lunch",
        "place_id",
        "url",
        "geometry/viewport/southwest/lat",
        "utc_offset",
        "permanently_closed",
        "geometry/viewport/southwest",
        "takeout",
        "delivery",
        "dine_in",
        "opening_hours",
        "wheelchair_accessible_entrance",
        "name",
        "review",
        "adr_address",
        "formatted_address",
        "serves_beer",
        "formatted_phone_number",
        "international_phone_number",
        "serves_brunch",
        "geometry/location",
        "geometry/viewport/southwest/lng",
        "user_ratings_total",
        "geometry/viewport",
        "serves_wine",
        "icon",
        "vicinity",
        "serves_dinner",
        "reviews",
        "geometry",
        "business_status",
        "geometry/viewport/northeast",
    ],
)
# Extract relevant data
result = place_details.get("result", {})

data = place_details.get("result", {})


# Flatten nested fields
def flatten_dict(d, parent_key="", sep="_"):
    """Recursively flattens a nested dictionary."""
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        elif isinstance(v, list):
            items.append((new_key, str(v)))  # Convert lists to string
        else:
            items.append((new_key, v))
    return dict(items)


flat_data = flatten_dict(data)


# Add random boolean for paymentOptions
flat_data["cardpayment"] = random.choice([True, False])
flat_data["cashpayment"] = random.choice([True, False])
flat_data["dog_friendly"] = random.choice([True, False])
flat_data["laptop_friendly"] = random.choice([True, False])

# Convert to DataFrame
df_place = pd.DataFrame([flat_data])
df_place

,address_components,adr_address,business_status,current_opening_hours_open_now,current_opening_hours_periods,current_opening_hours_weekday_text,editorial_summary_language,editorial_summary_overview,formatted_address,formatted_phone_number,...,url,user_ratings_total,utc_offset,vicinity,website,wheelchair_accessible_entrance,cardpayment,cashpayment,dog_friendly,laptop_friendly
0,"[{'long_name': '1600', 'short_name': '1600', '...","<span class=""street-address"">1600 Amphitheatre...",OPERATIONAL,False,"[{'close': {'date': '2025-03-31', 'day': 1, 't...",['Monday: 8:00\u202fAM\u2009–\u20095:00\u202fP...,en,"The global headquarters of Google, this campus...","1600 Amphitheatre Pkwy, Mountain View, CA 9404...",(650) 253-0000,...,https://maps.google.com/?cid=7793879817120868320,9634,-420,"1600 Amphitheatre Parkway, Mountain View",https://about.google/locations/?region=north-a...,True,False,False,True,False


In [38]:
location = (37.7749, -122.4194)  # Example: San Francisco, CA
radius = 1000  # 1000 meters (1km)
place_type = "restaurant"  # Search for restaurants

# Perform Nearby Search
places_result = gmaps.places_nearby(location=location, radius=radius, type=place_type)

# Display results
for place in places_result.get("results", []):
    print(f"📍 Name: {place.get('name', 'N/A')}")
    print(f"📍 Address: {place.get('vicinity', 'N/A')}")
    print(f"⭐ Rating: {place.get('rating', 'N/A')}")
    print("-" * 40)

📍 Name: Zuni Café
📍 Address: 1658 Market Street, San Francisco
⭐ Rating: 4.4
----------------------------------------
📍 Name: Absinthe Brasserie & Bar
📍 Address: 398 Hayes Street, San Francisco
⭐ Rating: 4.4
----------------------------------------
📍 Name: DNA Lounge
📍 Address: 375 11th Street, San Francisco
⭐ Rating: 4.1
----------------------------------------
📍 Name: Zeitgeist
📍 Address: 199 Valencia Street, San Francisco
⭐ Rating: 4.3
----------------------------------------
📍 Name: Suppenküche
📍 Address: 525 Laguna Street, San Francisco
⭐ Rating: 4.5
----------------------------------------
📍 Name: Brenda's French Soul Food
📍 Address: 652 Polk Street, San Francisco
⭐ Rating: 4.5
----------------------------------------
📍 Name: Hayes Street Grill
📍 Address: 320 Hayes Street, San Francisco
⭐ Rating: 4.2
----------------------------------------
📍 Name: AsiaSF
📍 Address: 201 9th Street, San Francisco
⭐ Rating: 4.5
----------------------------------------
📍 Name: Patxi's Pizza
📍 Addres

In [21]:
from src.apicall import get_places_details

In [ ]:
import requests
import re

def get_place_id(google_maps_url):
    """Extracts the Place ID from a Google Maps URL."""
    
    # Step 1: Expand the short URL if needed
    response = requests.get(google_maps_url, allow_redirects=True)
    expanded_url = response.url  # Get the final redirected URL
    
    # Step 2: Extract Place ID using regex
    match = re.search(r"place_id=([A-Za-z0-9_-]+)", expanded_url)
    
    if match:
        return match.group(1)  # Extract the Place ID
    else:
        return None  # Return None if no Place ID is found

# Example usage:
google_maps_url = "https://maps.app.goo.gl/QuDoni5xCfM1TqC56"
place_id = get_place_id(google_maps_url)

print("Extracted Place ID:", place_id)


In [24]:
# place_details=get_places_details(coordinate=(52.523249, 13.392378))
place_details=get_places_details(link="https://maps.app.goo.gl/QuDoni5xCfM1TqC56")
place_details

# ChIJ_UY9UudRqEcReQhTrUmBr4k


0x47a851db2026f155


ApiError: INVALID_REQUEST (Invalid request. Invalid 'placeid' parameter.)

In [23]:
place_details["name"]

0                     Radisson Collection Hotel, Berlin
1                                           Grill Royal
2                                     The Sixties Diner
3                                      Restaurant Nolle
4                                    Ishin Mittelstraße
5                             EINSTEIN Unter den Linden
6                                     Brechts Steakhaus
7                                         Cookies Cream
8                                               Tausend
9                                Restaurant Via Nova II
10                                    Berliner Republik
11                                   Kilkenny Irish Pub
12                                               Aposto
13                                    Ganymed Brasserie
14                                            Das Lemke
15                                   YOSOY TAPAS BERLIN
16                                            Borchardt
17                                    Schwarzwal